In [2]:
import pandas as pd
from functools import partial
from pathlib import Path
from sklearn.preprocessing import MultiLabelBinarizer
from collections import defaultdict, Counter

**Load nucleolar protein data with localCIDER and NARDINI scores**

In [ ]:
representations_45S_47S_df_w_cider_nardini = pd.read_csv("data/curated_data_for_modeling/localcider_features_w_nardini.tsv", sep="\t")
representations_45S_47S_df_w_cider_nardini

,Majority.protein.IDs,Reviewed,Protein ID,Sequence,Label,Length,length,FCR,NCPR,isoelectric_point,...,zsc_PROLINE_PROLINE,zsc_PROLINE_GLYCINE,zsc_GLYCINE_POLAR,zsc_GLYCINE_HYDROPHOBIC,zsc_GLYCINE_POSITIVE,zsc_GLYCINE_NEGATIVE,zsc_GLYCINE_AROMATIC,zsc_GLYCINE_ALANINE,zsc_GLYCINE_PROLINE,zsc_GLYCINE_GLYCINE
0,A0A087WPF7,reviewed,A0A087WPF7,MDGPTRGHGLRKKRRSRSQRDRERRSRAGLGTGAAGGIGAGRTRAP...,Nuclear Proteome,1261,1261.0,0.231562,0.019033,9.62500,...,1.793490,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000
1,A0A0A6YY25,reviewed,A0A0A6YY25,MCSPASSKILYRNPRFLRVAFLQLHHQQQSGVFCDALLQAEGEAVP...,Nuclear Proteome,723,723.0,0.255878,-0.045643,4.59375,...,-0.351355,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000
2,A0A0M3U1B0,reviewed,A0A0M3U1B0,MSTLQSLIIDGFHGNGFHMIEEYLQQKESHVPQKYNHLLLHHLDRL...,Nuclear Proteome,842,842.0,0.287411,-0.019002,6.12500,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000
3,A0A140LI88,reviewed,A0A140LI88,MENGAEASDCDSDETVIEGSVTENEPEDEELPWRRLLLNQDTTCRS...,Nuclear Proteome,1857,1857.0,0.260635,-0.014001,6.56250,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000
4,A0A1D9BZF0,reviewed,A0A1D9BZF0,MDSGSSSSSSSSGSSSGSCSTSGSGSTSGSSTTSSSSSSSSSSSSS...,Nuclear Proteome,507,507.0,0.378698,-0.106509,4.15625,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2.08136
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5978,Q9WVA4,reviewed,Q9WVA4,MANRGPSYGLSREVQQKIEKQYDADLEQILIQWITTQCREDVGQPQ...,ITS2,199,199.0,0.221106,0.010050,8.75000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000
5979,Q8CGP4;C0HKE9;C0HKE8;C0HKE7;C0HKE6;C0HKE5;C0HK...,reviewed,C0HKE9,MSGRGKQGGKARAKAKTRSSRAGLQFPVGRVHRLLRKGNYSERVGA...,ITS2,130,130.0,0.269231,0.130769,11.37500,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.15812
5980,P63242,reviewed,P63242,MADDLDFETGDAGASATFPMQCSALRKNGFVVLKGRPCKIVEMSTS...,ITS2,154,154.0,0.279221,-0.045455,4.81250,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000
5981,Q6NXY9,reviewed,Q6NXY9,MAGNKGRGRAAYTFNIEAVGFSRGEKLPDVVLKPPPLFPDTDYKPV...,ITS2,223,223.0,0.452915,-0.112108,4.15625,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000


**Import the mouse proteome from Uniprot**

In [7]:
uniprot_df = pd.read_csv("data/uniprot_data/uniprotkb_taxonomy_id_10090_2025_03_25.tsv", sep="\t")   
uniprot_df

/tmp/ipykernel_558039/3283731104.py:1: DtypeWarning: Columns (10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  uniprot_df = pd.read_csv("/ceph/hpc/home/novljanj/data_storage/projects/nucleolus_enriched_proteins/data/uniprot_data/uniprotkb_taxonomy_id_10090_2025_03_25.tsv", sep="\t")


,Entry,Reviewed,Entry Name,Protein names,Gene Names,Organism,Length,Compositional bias,Mass,Sequence,...,Redox potential,Region,Ensembl,Coiled coil,Domain [CC],Domain [FT],Motif,Repeat,Zinc finger,Gene Ontology (cellular component)
0,A0A075F5C6,unreviewed,A0A075F5C6_MOUSE,Heat shock factor 1 (Heat shock transcription ...,Hsf1,Mus musculus (Mouse),531,"COMPBIAS 343..355; /note=""Polar residues""; /ev...",57879,MDLAVGPGAAGPSNVPAFLTKLWTLVSDPDTDALICWSPSGNSFHV...,...,NaN,"REGION 272..327; /note=""Disordered""; /evidence...",ENSMUST00000228371.2;,NaN,NaN,"DOMAIN 57..81; /note=""HSF-type DNA-binding""; /...",NaN,NaN,NaN,centrosome [GO:0005813]; cytosol [GO:0005829];...
1,A0A087WPF7,reviewed,AUTS2_MOUSE,Autism susceptibility gene 2 protein homolog,Auts2 Kiaa0442,Mus musculus (Mouse),1261,"COMPBIAS 8..17; /note=""Basic residues""; /evide...",138920,MDGPTRGHGLRKKRRSRSQRDRERRSRAGLGTGAAGGIGAGRTRAP...,...,NaN,"REGION 1..88; /note=""Disordered""; /evidence=""E...",ENSMUST00000161226.11 [A0A087WPF7-1];ENSMUST00...,NaN,DOMAIN: The Pro-rich region is important for t...,NaN,NaN,NaN,NaN,cytoplasm [GO:0005737]; cytoskeleton [GO:00058...
2,A0A087WPT2,unreviewed,A0A087WPT2_MOUSE,Prostaglandin-endoperoxide synthase 2,Ptgs2,Mus musculus (Mouse),62,NaN,7199,MSTGFDQYKCDCTRTGFYGENCTTPEFLTRIKLLLKPTPNTVHYIL...,...,NaN,NaN,ENSMUST00000190784.2;,NaN,NaN,"DOMAIN 1..23; /note=""EGF-like""; /evidence=""ECO...",NaN,NaN,NaN,NaN
3,A0A087WRK1,unreviewed,A0A087WRK1_MOUSE,"Predicted gene, 20814 (Predicted gene, 20855) ...",Gm20905 Gm20814 Gm20835 Gm20850 Gm20855 Gm2086...,Mus musculus (Mouse),222,NaN,25620,MRRMALKKLKVIPKEGYLLLLDFDDEDDDIKVSEEALSEVKSPAFD...,...,NaN,"REGION 39..64; /note=""Disordered""; /evidence=""...",ENSMUST00000185240.2;ENSMUST00000185245.2;ENSM...,NaN,NaN,"DOMAIN 112..220; /note=""XLR/SYCP3/FAM9""; /evid...",NaN,NaN,NaN,cytoplasm [GO:0005737]; nucleus [GO:0005634]
4,A0A087WRT4,unreviewed,A0A087WRT4_MOUSE,FAT atypical cadherin 1,Fat1,Mus musculus (Mouse),4602,"COMPBIAS 4258..4267; /note=""Polar residues""; /...",507531,MGRHLTLLLLLLLFLQQFGDSDGSQRLEPTPPIQFTHFQYNVTVHE...,...,NaN,"REGION 4256..4292; /note=""Disordered""; /eviden...",ENSMUST00000189017.8;,NaN,NaN,"DOMAIN 36..150; /note=""Cadherin""; /evidence=""E...",NaN,NaN,NaN,apical plasma membrane [GO:0016324]; cell junc...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87487,Z4YN82,unreviewed,Z4YN82_MOUSE,Syntaphilin,Snph,Mus musculus (Mouse),87,NaN,9364,MAMSLQGSRRASAGSRRRTSPPVSVRDAYGTSSLSSSSNSGSCKGS...,...,NaN,"REGION 1..74; /note=""Disordered""; /evidence=""E...",ENSMUST00000137936.2;,NaN,NaN,NaN,NaN,NaN,NaN,membrane [GO:0016020]
87488,Z4YN86,unreviewed,Z4YN86_MOUSE,Translocase of inner mitochondrial membrane 22,Timm22,Mus musculus (Mouse),116,NaN,12689,MAATAPKAGGSAPEAAGSAEAPLQYSLLLQYLVGDKRQPRLLEPGS...,...,NaN,NaN,ENSMUST00000152183.2;,NaN,NaN,NaN,NaN,NaN,NaN,NaN
87489,Z4YN92,unreviewed,Z4YN92_MOUSE,A kinase anchor protein 17B,Akap17b,Mus musculus (Mouse),406,"COMPBIAS 24..52; /note=""Basic and acidic resid...",47765,MFDTTKHFSEGAIQRRNQERLKLQELEEERKKEKKREEEVAERKRK...,...,NaN,"REGION 24..90; /note=""Disordered""; /evidence=""...",ENSMUST00000133980.2;,NaN,NaN,NaN,NaN,NaN,NaN,NaN
87490,Z4YN97,unreviewed,Z4YN97_MOUSE,Adenylate kinase 1,Ak1,Mus musculus (Mouse),89,NaN,9541,MEEKLKKAKIIFVVGGPGSGKGTQCEKIVQKYGYTHLSTGDLLRAE...,...,NaN,NaN,ENSMUST00000156578.8;,NaN,NaN,NaN,NaN,NaN,NaN,cytoplasm [GO:0005737]


**Parse the domains, regions and compositional bias annotation over protein sequence into the percent of protein**

In [ ]:
representations_45S_47S_df_w_cider_nardini_with_uniprot = representations_45S_47S_df_w_cider_nardini.merge(uniprot_df, left_on="Protein ID", right_on="Entry", how="inner")

import pandas as pd, re

FEATURE_RE  = re.compile(r'(\d+)\.\.(\d+).*?/note="([^"]+)"')  
NOTE_CLEAN  = re.compile(r'\s*\d+$')                          

def feature_matrix(df: pd.DataFrame,
                   feature_col: str = "Domain [FT]",
                   length_col: str  = "Length_x") -> pd.DataFrame:

    unique_notes = set()
    for cell in df[feature_col].fillna(""):
        for _, _, raw_note in FEATURE_RE.findall(cell):
            unique_notes.add(NOTE_CLEAN.sub("", raw_note).strip())
    note_list = sorted(unique_notes)

    matrix = pd.DataFrame(0.0, index=df.index, columns=note_list)

    for idx, row in df[[feature_col, length_col]].iterrows():
        cell, prot_len = row[feature_col], row[length_col]

        if pd.isna(cell) or cell == "" or pd.isna(prot_len):
            continue

        prot_len = float(prot_len)

        for start, end, raw_note in FEATURE_RE.findall(cell):
            span       = int(end) - int(start) + 1
            clean_note = NOTE_CLEAN.sub("", raw_note).strip()
            matrix.at[idx, clean_note] += span / prot_len

    return matrix

with_uni = representations_45S_47S_df_w_cider_nardini_with_uniprot
base     = representations_45S_47S_df_w_cider_nardini

for col in ["Domain [FT]", "Region", "Compositional bias"]:
    matrix = feature_matrix(with_uni, feature_col=col)
    base = pd.concat([base, matrix], axis=1)

representations_45S_47S_df_w_nardini_with_uniprot_parsed = base 

print(f"Added {len(representations_45S_47S_df_w_nardini_with_uniprot_parsed.columns.difference(representations_45S_47S_df_w_cider_nardini.columns))} columns to the dataframe")

representations_45S_47S_df_w_nardini_with_uniprot_parsed

Added 3430 columns to the dataframe


,Majority.protein.IDs,Reviewed,Protein ID,Sequence,Label,Length,length,FCR,NCPR,isoelectric_point,...,phosphorylated FFAT motif binding,preferentially binds ssDNA,siRNA-binding,tRNA-binding,zUBD/ZHA,Acidic residues,Basic and acidic residues,Basic residues,Polar residues,Pro residues
0,A0A087WPF7,reviewed,A0A087WPF7,MDGPTRGHGLRKKRRSRSQRDRERRSRAGLGTGAAGGIGAGRTRAP...,Nuclear Proteome,1261,1261.0,0.231562,0.019033,9.62500,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.153846,0.036479,0.031721,0.00793
1,A0A0A6YY25,reviewed,A0A0A6YY25,MCSPASSKILYRNPRFLRVAFLQLHHQQQSGVFCDALLQAEGEAVP...,Nuclear Proteome,723,723.0,0.255878,-0.045643,4.59375,...,0.0,0.0,0.0,0.0,0.0,0.015214,0.041494,0.000000,0.047026,0.00000
2,A0A0M3U1B0,reviewed,A0A0M3U1B0,MSTLQSLIIDGFHGNGFHMIEEYLQQKESHVPQKYNHLLLHHLDRL...,Nuclear Proteome,842,842.0,0.287411,-0.019002,6.12500,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.067696,0.020190,0.000000,0.00000
3,A0A140LI88,reviewed,A0A140LI88,MENGAEASDCDSDETVIEGSVTENEPEDEELPWRRLLLNQDTTCRS...,Nuclear Proteome,1857,1857.0,0.260635,-0.014001,6.56250,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.036080,0.007001,0.043619,0.00000
4,A0A1D9BZF0,reviewed,A0A1D9BZF0,MDSGSSSSSSSSGSSSGSCSTSGSGSTSGSSTTSSSSSSSSSSSSS...,Nuclear Proteome,507,507.0,0.378698,-0.106509,4.15625,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.424063,0.055227,0.000000,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5978,Q9WVA4,reviewed,Q9WVA4,MANRGPSYGLSREVQQKIEKQYDADLEQILIQWITTQCREDVGQPQ...,ITS2,199,199.0,0.221106,0.010050,8.75000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.00000
5979,Q8CGP4;C0HKE9;C0HKE8;C0HKE7;C0HKE6;C0HKE5;C0HK...,reviewed,C0HKE9,MSGRGKQGGKARAKAKTRSSRAGLQFPVGRVHRLLRKGNYSERVGA...,ITS2,130,130.0,0.269231,0.130769,11.37500,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.100000,0.000000,0.00000
5980,P63242,reviewed,P63242,MADDLDFETGDAGASATFPMQCSALRKNGFVVLKGRPCKIVEMSTS...,ITS2,154,154.0,0.279221,-0.045455,4.81250,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.00000
5981,Q6NXY9,reviewed,Q6NXY9,MAGNKGRGRAAYTFNIEAVGFSRGEKLPDVVLKPPPLFPDTDYKPV...,ITS2,223,223.0,0.452915,-0.112108,4.15625,...,0.0,0.0,0.0,0.0,0.0,0.197309,0.121076,0.000000,0.000000,0.00000


In [9]:
representations_45S_47S_df_w_nardini_with_uniprot_parsed.to_csv("data/curated_data_for_modeling/localcider_features_w_nardini_w_uniprot.tsv", sep="\t", index=False)